## Natural language inference

### Contradictory dear watson
***

Project: https://www.kaggle.com/competitions/contradictory-my-dear-watson

### Model: distil_bert_base_multi

https://huggingface.co/distilbert/distilbert-base-multilingual-cased

__Objetivo__: testear el modelo entendiendo su funcionamiento, bias y limitaciones

This model is a distilled version of the BERT base multilingual model. The code for the distillation process can be found here. This model is cased: it does make a difference between english and English.

The model is trained on the concatenation of Wikipedia in 104 different languages listed here. The model has 6 layers, 768 dimension and 12 heads, totalizing 134M parameters (compared to 177M parameters for mBERT-base). On average, this model, referred to as DistilmBERT, is twice as fast as mBERT-base

**Importar Librerias**

In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
from tensorflow import keras
import keras_nlp
import seaborn as sns
import matplotlib.pyplot as plt
import os

print("TensorFlow version:", tf.__version__)
print("KerasNLP version:", keras_nlp.__version__)

E0000 00:00:1725577179.118599      13 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:479
D0905 22:59:39.126897762      13 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D0905 22:59:39.126912280      13 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D0905 22:59:39.126915773      13 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D0905 22:59:39.126918177      13 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D0905 22:59:39.126920478      13 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defa

TensorFlow version: 2.16.1
KerasNLP version: 0.14.4


__Conf__

In [2]:
try:
    # detect and init the TPU
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
    print("All devices: ", tf.config.list_logical_devices('TPU'))
except ValueError:
    strategy = tf.distribute.get_strategy()  # default strategy if no TPU available

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1725577193.421930      13 service.cc:145] XLA service 0x55fd9c166de0 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1725577193.421985      13 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1725577193.421989      13 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1725577193.421992      13 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1725577193.421996      13 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1725577193.421998      13 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1725577193.422001      13 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1725577193.422004      13 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1725577193.422007      13 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [3]:
RESULT_DICT = {
    0 : "entailment",
    1 : "neutral",
    2 : "contradiction"
}

__Data__

In [4]:
df_train = pd.read_csv('/kaggle/input/contradictory-my-dear-watson/train.csv')
df_test = pd.read_csv('/kaggle/input/contradictory-my-dear-watson/test.csv')

In [5]:
def display_pair_of_sentence(x):
    print( "Premise : " + x['premise'])
    print( "Hypothesis: " + x['hypothesis'])
    print( "Language: " + x['language'])
    print( "Label: " + str(x['label']))
    print()

df_train.head(5).apply(lambda x : display_pair_of_sentence(x), axis=1)

df_train.shape

Premise : and these comments were considered in formulating the interim rules.
Hypothesis: The rules developed in the interim were put together with these comments in mind.
Language: English
Label: 0

Premise : These are issues that we wrestle with in practice groups of law firms, she said. 
Hypothesis: Practice groups are not permitted to work on these issues.
Language: English
Label: 2

Premise : Des petites choses comme celles-là font une différence énorme dans ce que j'essaye de faire.
Hypothesis: J'essayais d'accomplir quelque chose.
Language: French
Label: 0

Premise : you know they can't really defend themselves like somebody grown uh say my age you know yeah
Hypothesis: They can't defend themselves because of their age.
Language: English
Label: 0

Premise : ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสดงออกและได้เล่นหลายบทบาทไปพร้อมกัน ๆ อาจช่วยให้เด็กจับความคล้ายคลึงและความแตกต่างระหว่างผู้คนในด้านความปรารถนา ความเชื่อ และความรู้สึกได้
Hypothesis: เด็กสามารถเห็นได้ว่าชาติพันธุ

(12120, 6)

In [6]:
df_train["premise_length"] = df_train["premise"].apply(lambda x : len(x))
df_train["hypothesis_length"] = df_train["hypothesis"].apply(lambda x : len(x))
df_train[["hypothesis_length", "premise_length"]].describe()

,hypothesis_length,premise_length
count,12120.000000,12120.000000
mean,53.892327,107.373185
std,25.302358,71.089954
min,4.000000,4.000000
25%,36.000000,55.000000
50%,51.000000,96.000000
75%,67.000000,146.000000
max,276.000000,967.000000


__Train Model__

In [7]:
VALIDATION_SPLIT = 0.2
TRAIN_SIZE = int(df_train.shape[0]*(1-VALIDATION_SPLIT))
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

In [8]:
def split_labels(x, y):
    return (x[0], x[1]), y


training_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            df_train[['premise','hypothesis']].values,
            keras.utils.to_categorical(df_train['label'], num_classes=3)
        )
    )
)

train_dataset = training_dataset.take(TRAIN_SIZE)
val_dataset = training_dataset.skip(TRAIN_SIZE)

# Apply the preprocessor to every sample of train, val and test data using `map()`.
# [`tf.data.AUTOTUNE`](https://www.tensorflow.org/api_docs/python/tf/data/AUTOTUNE) and `prefetch()` are options to tune performance, see
# https://www.tensorflow.org/guide/data_performance for details.

train_preprocessed = train_dataset.map(split_labels, tf.data.AUTOTUNE).batch(BATCH_SIZE, drop_remainder=True).cache().prefetch(tf.data.AUTOTUNE)
val_preprocessed = val_dataset.map(split_labels, tf.data.AUTOTUNE).batch(BATCH_SIZE, drop_remainder=True).cache().prefetch(tf.data.AUTOTUNE)

In [9]:
# Load a BERT model.
with strategy.scope():
    classifier = keras_nlp.models.DistilBertClassifier.from_preset("distil_bert_base_multi", num_classes=3)

    # in distributed training, the recommendation is to scale batch size and learning rate with the numer of workers.
    classifier.compile(optimizer=keras.optimizers.Adam(1e-5*strategy.num_replicas_in_sync),
                       loss=keras.losses.CategoricalCrossentropy(from_logits=True),
                       metrics=['accuracy'])
    
    classifier.summary()

I0000 00:00:1725577202.797353      13 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Preprocessor: "distil_bert_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ distil_bert_tokenizer (DistilBertTokenizer)        │                                             119,547 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "distil_bert_classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ distil_bert_backbone          │ (None, None, 768)         │     134,734,080 │ padding_mask[0][0],        │
│ (DistilBertBackbone)          │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ get_item (GetItem)            │ (None, 768)               │               0 │ distil_bert_backbone[0][0] │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pooled_dense (Dense)          │ (None, 768)               │         590,592 │ get_item[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ output_dropout (Dropout)      │ (None, 768)               │               0 │ pooled_dense[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ logits (Dense)                │ (None, 3)                 │           2,307 │ output_dropout[0][0]       │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 135,326,979 (516.23 MB)

 Trainable params: 135,326,979 (516.23 MB)

 Non-trainable params: 0 (0.00 B)

__Fine tunning distil bert__

In [10]:
EPOCHS=10
history = classifier.fit(train_preprocessed,
                         epochs=EPOCHS,
                         validation_data=val_preprocessed
                        )

Epoch 1/10


2024-09-05 23:00:41.869199: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node StatefulPartitionedCall.
I0000 00:00:1725577243.110321     808 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(cbde905cd30e45e:0:0), session_name()
I0000 00:00:1725577295.936578     808 tpu_compile_op_common.cc:245] Compilation of cbde905cd30e45e:0:0 with session name  took 52.82621072s and succeeded
I0000 00:00:1725577296.077350     808 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(cbde905cd30e45e:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_iterator_10397702090445519876", property.function_library_fingerprint = 6504866068266459693, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1,

75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step - accuracy: 0.3884 - loss: 1.0737

2024-09-05 23:01:54.189836: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.
I0000 00:00:1725577314.579050     828 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(98217fede1cb2298:0:0), session_name()
I0000 00:00:1725577317.216314     828 tpu_compile_op_common.cc:245] Compilation of 98217fede1cb2298:0:0 with session name  took 2.63722141s and succeeded
I0000 00:00:1725577317.234076     828 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(98217fede1cb2298:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_iterator_759579976358246455", property.function_library_fingerprint = 9189351796091401269, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topo

75/75 ━━━━━━━━━━━━━━━━━━━━ 85s 293ms/step - accuracy: 0.3894 - loss: 1.0730 - val_accuracy: 0.5729 - val_loss: 0.9399
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 19s 233ms/step - accuracy: 0.5855 - loss: 0.9143 - val_accuracy: 0.6250 - val_loss: 1.0085
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 19s 241ms/step - accuracy: 0.7010 - loss: 0.7310 - val_accuracy: 0.5660 - val_loss: 1.1464
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 19s 234ms/step - accuracy: 0.7794 - loss: 0.5585 - val_accuracy: 0.5903 - val_loss: 1.1386
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 19s 234ms/step - accuracy: 0.8508 - loss: 0.4062 - val_accuracy: 0.5799 - val_loss: 1.3967
Epoch 6/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 19s 233ms/step - accuracy: 0.9041 - loss: 0.2882 - val_accuracy: 0.5868 - val_loss: 1.7065
Epoch 7/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 19s 243ms/step - accuracy: 0.9150 - loss: 0.2354 - val_accuracy: 0.5556 - val_loss: 1.8179
Epoch 8/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 19s 234ms/step - accuracy: 0.9160 - loss: 0.2158 - val_accuracy: 0.611